In [1]:
import numpy as np

from utils.base import rotation_matrix, make_exp_matrix, make_screw_axis_matrix

# Inverse Kinematics


### Revise 

![ Inverse kinematics of a 2R planar open chain.](./assets/fig6_1.png)



The forward kinematics can be expressed as:


$$
\begin{bmatrix}
x \\
y
\end{bmatrix}
=
\begin{bmatrix}
L_1 \cos \theta_1 + L_2 \cos(\theta_1 + \theta_2) \\
L_1 \sin \theta_1 + L_2 \sin(\theta_1 + \theta_2)
\end{bmatrix}
$$


Assuming $ L_1 > L_2$  ,  the set of reachable points is an annulus of inner radius $L_1 - L_2$ and outer radius $L_1 + L_2$ . In this workspace , any given point will be either one / two solutions


Finding an explicit solution $ ( \theta_1 , \theta_2)$ for given (x,y), using law of cosines,

$$
c^2 = a^2 + b^2 - 2 a b cos(C)
$$

$$
L_1 ^2 + L_2^2 - 2 L_1 L_2 cos(\beta) = x^2 + y^2
$$


,where
$$
\beta = \cos^{-1} \left( \frac{L_1^2 + L_2^2 - x^2 - y^2}{2 L_1 L_2} \right)
$$
and 
$$
\alpha = \cos^{-1} \left( \frac{x^2 + y^2 + L_1^2 - L_2^2}{2 L_1 \sqrt{x^2 + y^2}} \right)
$$

The angle $ \gamma = \mathrm{atan2}(y, x)$. With:

righty solution:

$$
\theta_1 = \gamma - \alpha , \theta_2 = \pi - \beta
$$

left solution:

$$
\theta_1 = \gamma + \alpha , \theta_2 = \beta - \pi
$$

## Inverse kinematics of spatial open chains with six degress of freedom



Forward kinematics:

$$
T(\boldsymbol{\theta}) = e^{[S_1]\theta_1} e^{[S_2]\theta_2} e^{[S_3]\theta_3} e^{[S_4]\theta_4} e^{[S_5]\theta_5} e^{[S_6]\theta_6} M
$$

Given some end-effector frame $ X \in SE(3) $, the inverse kinematics problem is to
find solutions $ \theta \in \mathbb{R}^6 $ satisfying $ T(\theta) = X $.
In the following subsections, we derive analytic inverse kinematic solutions
for the PUMA and Stanford arms.


### example 

![ Inverse kinematics of a 6R PUMA-type arm.](./assets/fig6_2.png)

In [ ]:
d1 = 0

px, py, pz = 1.0, 2.0, 3.0

a1, a2, a3 = 0.1, 0.2, 0.3


In [ ]:
if d1 == 0: # no offset


    # Calculate the angles theta1 and theta2
    theta1 = np.arctan2(py, px)
    # or 
    # theta1 = np.arctan2(py,px)+np.pi if px < 0 else np.arctan2(py,px)
    # as long as px, py != 0 both are valid

    # when px = py = 0
    # theta1 has infinite solutions


![A 6R PUMA-type arm with a shoulder offset.](./assets/fig6_3.png)

In [ ]:
if d1 != 0:
    # in general there will be 2 solutions for theta1 , right and left
    r = px**2 + py**2
    phi = np.arctan2(py,px)
    alpha = np.arctan2(d1, np.sqrt(r - d1**2))

    theta1 = phi - alpha
    # or
    # theta1 = pi + atan2(py,px) +artan2(-sqrt(px^2 + py^2 - d1^2 ), d1) 

In [ ]:
theta3 = np.acos(r**2 - d1**2 + pz**2 - a2**2 - a3**2) / (2*a2*a3)
# or
# theta3 = pi - acos(px^2 + py^2  - d1^2 + pz^2 - a2^2 - a3^2) / (2*a2*a3)

D = np.cos(theta3)
# theta3 = artan2(+/-np.sqrt(1 - D**2), D)

theta2 = np.arctan2(pz, np.sqrt(r**2 - d1**2))- np.arctan2(a3*np.sin(theta3), a2 + a3*np.cos(theta3))
# or
# theta2 = pi - atan2(pz, sqrt(px^2 + py^2 - d1^2)) - atan2(a3*sin(theta3), a2 + a3*cos(theta3))

### next setion
finding $(\theta_4, \theta_5, \theta_6)$ given the end-effector orientation . After finding $(\theta_1, \theta_2, \theta_3)$ , the forward kinematics can be restructured as :

$$
e^{-[S_3]\theta_3} e^{-[S_2]\theta_2} e^{-[S_1]\theta_1} X M^{-1} = e^{[S_4]\theta_4} e^{[S_5]\theta_5} e^{[S_6]\theta_6}
$$

where the right-hand size is now known and the $w_i$ components of $S_{4,5,6}$

In [ ]:
w4 = np.array([0, 0, 1])
w5 = np.array([0, 1, 0])
w6 = np.array([1, 0, 0])



R = rotation_matrix('z', theta4) @ rotation_matrix('y', theta5) @ rotation_matrix('x', theta6)

### Stanford-Type Arm


RRPRRR arm


![The first three joints of a Stanford-type arm.](./assets/fig6_6.png)





In [ ]:
r = np.sqrt(px**2 + py**2)
s = pz - d1

theta1 = np.arctan2(py, px) # if px == py == 0
theta2 = np.arctan2(r,s)

# theta1 = np.arctan2(py, px) + np.pi second solution for theta1
# theta2 = pi - np.arctan2(r,s) second solution for theta2 

# translation distance
theta3 = np.sqrt(r**2 + s**2) - a2 


### Numerical Inverse Kinematics

#### Newton-Raphson Method

To solve the equation $g(\theta) = 0$ , assuming $\theta^(0)$ is an initial guess.

$$
g(\theta) = g(\theta^0) + \frac{\partial g}{\partial \theta}(\theta^0)(\theta - \theta^0) + \text{higher-order terms (h.o.t.)}
$$

keeping only the terms up to first order

$$
\theta = \theta^{0} - (\frac{\partial g}{\partial \theta}(\theta^0))^{-1} g(\theta^0)
$$

which can also phrase as an iterative guess

$$
\theta^{k+1} = \theta^{k} - (\frac{\partial g}{\partial \theta}(\theta^k))^{-1} g(\theta^k)
$$

Repeat until $
\frac{|g(\theta_k) - g(\theta_{k+1})|}{|g(\theta_k)|} \leq \epsilon
$. 


Applied for g is multi dimensional

e.g :
$$
\frac{\partial g}{\partial \theta}(\theta) =
\begin{bmatrix}
\frac{\partial g_1}{\partial \theta_1}(\theta) & \cdots & \frac{\partial g_1}{\partial \theta_n}(\theta) \\
\vdots & \ddots & \vdots \\
\frac{\partial g_n}{\partial \theta_1}(\theta) & \cdots & \frac{\partial g_n}{\partial \theta_n}(\theta)
\end{bmatrix}
\in \mathbb{R}^{n \times n}.
$$




### Numerical Inverse Kinematic Algorithm

express end-effecotr frame using a coodinate vector $x$ governed by the forward kineatics $x=f(\theta)$

let $x_d$ be the desired end-effector coordintaes. 

$$
g(\theta_d) = x_d - f(\theta_d) = 0
$$

Guess an initial guess $\theta^0$ which is "close to" a solution:

$$
x_d = f(\theta_d) = f(\theta_0) + 
\underbrace{\frac{\partial f}{\partial \theta}(\theta_0)}_{J(\theta_0)}
\underbrace{(\theta_d - \theta_0)}_{\Delta \theta} 
+ \text{h.o.t.}
$$

$$
J(\theta_0) \theta_d = x_d - f(\theta^0)
$$

if $J(\theta_0)$ is square and invertiable 

$$
\theta_d = J^{-1}(\theta_0) x_d - f(\theta^0)
$$

IF $J$ is not invertible , either:
- not square 
- singular

then $J^{-1}$ does not exist Then use:

$$
y^* = J^\dagger z
$$


falls into two categories:

- the solution satifies above equation for any solution . then $y^*$ minimizes the two-norm

- IF no y satisfy . then $y^*$ minimises the two-norm of the error


IF $J$ is full rank( rank $m$ for $n > m$) the robot is not at a singularity

$$
J^\dagger = J^T(J J^T)^{-1} \text{if n > m}
$$
$$
J^\dagger = (J J^T)^{-1} J^T \text{if n < m}





In [ ]:
def NewRap(xd, theta ,epilson , f , J):
    i = 0
    e = xd - f(theta)
    while e > epilson:
        theta = theta + J(theta) * 0.1
        e = xd - f(theta)
        i += 1
    return theta, i, e

In [ ]:
def body_twist(theta, Tsd):
    i = 0
    Vb = np.log(np.linalg.pinv(Tsb(theta)))

![(Left) A 2R robot. (Right) The goal is to find the joint angles yielding the
end-effector frame {goal} corresponding to θ1 = 30◦
and θ2 = 90◦
. The initial guess is
(0◦
, 30◦
). After one Newton–Raphson iteration, the calculated joint angles are
(34.23◦
, 79.18◦
). The screw axis that takes the initial frame to the goal frame (by means
of the curved dashed line) is also indicated.](./assets/fig6_8.png)

In [4]:

# finding the joint angles that place the tip of the robot at (x,y) = (0.366m , 1.366m ), which corresponds to 
# theta_d = (30 , 90)

Tsd = np.array([
    [-0.5 , -0.866, 0 , 0.366],
    [0.866, -0.5, 0 , 1.366],
    [0, 0, 1 , 0],
    [0, 0, 0 , 1]
])

M = np.array([
    [1, 0, 0, 2],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

B1_screw = np.array([0 , 0, 1, 0 , 2, 0])
B2_screw = np.array([0 , 0, 1, 0 , 1, 0])

theta_0 = np.array([0 ,  np.pi/6]) # 30 degrees = pi/6 rad
e_w  = 0.001 # rad / 0.0573 deg
e_v = 10^-4 # m

B1 = make_screw_axis_matrix(B1_screw[:3], B1_screw[3:])   
B2 = make_screw_axis_matrix(B2_screw[:3], B2_screw[3:])

# # Jacobian matrix
T01 = make_exp_matrix(B1, theta_0[0])
T12 = make_exp_matrix(B2, theta_0[1])
# T02 = T01 @ T12

T02 = M @ T01 @ T12 
print(T02)

[[ 0.8660254 -0.5        0.         1.8660254]
 [ 0.5        0.8660254  0.         0.5      ]
 [ 0.         0.         1.         0.       ]
 [ 0.         0.         0.         1.       ]]


In [ ]:
def Tsb(theta):
    T01 = make_exp_matrix(B1, theta[0])
    T12 = make_exp_matrix(B2, theta[1])
    T02 = M @ T01 @ T12 
    return T02
def NewRap(xd, theta ,epilson_w , epilson_v , Tsb , Tsd , J):
    i = 0
    Vb = np.log(np.linalg.pinv(Tsb(theta)) @ Tsd)
    while np.linalg.norm(Vb[:3]) > epilson_w or np.linalg.norm(Vb[3:]) > epilson_v:
        theta = theta + J(theta) * Vb
        i += 1
    return theta, i, Vb

In [7]:
!pip install modern_robotics

In [10]:
import numpy as np
import modern_robotics as mr

# Define the given matrices and parameters
Tsd = np.array([
    [-0.5, -0.866, 0, 0.366],
    [0.866, -0.5, 0, 1.366],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

M = np.array([
    [1, 0, 0, 2],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

B1 = np.array([0, 0, 1, 0, 2, 0])
B2 = np.array([0, 0, 1, 0, 1, 0])
Blist = np.vstack((B1, B2)).T  # 6x2 matrix

# Initial guess: theta0 = [0, 30 degrees], convert 30 degrees to radians
theta = np.array([0, np.pi/6])

# Error tolerances
eps_omega = 0.001  # rad
eps_v = 1e-4       # m

# Iteration parameters
max_iter = 4
i = 0
converged = False

print("Iteration | theta1 (rad) | theta2 (rad) | x (m) | y (m) | Vb | norm_wb (rad) | norm_vb (m)")

while i < max_iter and not converged:
    # Compute forward kinematics T_sb(theta_i)
    Tsb = mr.FKinBody(M, Blist, theta)
    
    # Extract x, y from T_sb translation
    x, y = Tsb[0, 3], Tsb[1, 3]
    
    # Compute T_sb^{-1} * T_sd
    Tsb_inv = mr.TransInv(Tsb)
    T_error = np.dot(Tsb_inv, Tsd)
    
    # Compute body velocity V_b using matrix logarithm
    se3mat = mr.MatrixLog6(T_error)  # Returns a 4x4 matrix
    Vb = mr.se3ToVec(se3mat)  # Convert to 6x1 vector
    
    # Extract angular and linear velocities
    omega_b = Vb[0:3]
    v_b = Vb[3:6]
    
    # Compute norms
    norm_wb = np.linalg.norm(omega_b)
    norm_vb = np.linalg.norm(v_b)
    
    # Print iteration data
    display_theta = np.degrees(theta)  # Convert to degrees for display
    print(f"{i:9d} | {display_theta[0]:12.6f} | {display_theta[1]:12.6f} | {x:5.3f} | {y:5.3f} | {Vb} | {norm_wb:12.6f} | {norm_vb:12.6f}")
    
    # Check convergence
    if norm_wb <= eps_omega and norm_vb <= eps_v:
        converged = True
        print(f"\nConverged after {i+1} iterations")
        print(f"Final theta: {theta} radians")
        break
    
    # Compute body Jacobian J_b(theta_i)
    Jb = mr.JacobianBody(Blist, theta)
    
    # Compute pseudoinverse of Jacobian
    Jb_pinv = np.linalg.pinv(Jb)
    
    # Update theta
    theta = theta + np.dot(Jb_pinv, Vb)
    
    i += 1

if not converged:
    print(f"\nDid not converge after {max_iter} iterations")
    print(f"Final theta: {theta} radians")

Iteration | theta1 (rad) | theta2 (rad) | x (m) | y (m) | Vb | norm_wb (rad) | norm_vb (m)
        0 |     0.000000 |    30.000000 | 1.866 | 0.500 | [0.         0.         1.57077447 0.4978665  1.85829122 0.        ] |     1.570774 |     1.923829
        1 |    34.233692 |    79.178180 | 0.429 | 1.480 | [ 0.          0.          0.11499553 -0.0735971   0.10804896  0.        ] |     0.114996 |     0.130733
        2 |    29.979533 |    90.221464 | 0.363 | 1.364 | [ 0.          0.         -0.00349531  0.00034795 -0.00347404  0.        ] |     0.003495 |     0.003491
        3 |    29.999467 |    90.001912 | 0.366 | 1.366 | [ 0.00000000e+00  0.00000000e+00 -1.13597272e-05  3.35908487e-09
  1.06401061e-05  0.00000000e+00] |     0.000011 |     0.000011

Converged after 4 iterations
Final theta: [0.52358947 1.57082969] radians
